In [43]:
import pandas as pd
import numpy as np
import math as mt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

dataset = pd.read_csv("cleaned_data.csv")
df = dataset
df
# Define and compile the CNN model
# model = Sequential()
# model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
# model.add(Flatten())
# model.add(Dense(1))

# model.compile(optimizer='adam', loss='mse')

# # Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# # Make predictions
# predictions = model.predict(X_test)

,Date,Time,Eqp1 Load(%),Eqp1 Pressure1(bar),Eqp1 Pressure2(bar),Eqp1 Supply Of oil Pressure(bar),Eqp1 oil filter variations Presssure(bar),Eqp1 Temp1(degree),Eqp1 Temp1 working,Eqp1 Temp2(degree),...,Eqp2 Incoming Temp(degree),Eqp2 Outgoiing Temp(degree),Eqp3 cold water flow (m/hr),Eqp3 Incoming Pressure(bar),Eqp3 Outgoing Pressure(bar),Eqp3 Variation in Pressure(bar),Eqp3 Incoming Temp(degree),Eqp3 Incoming Temp Working,Eqp3 Outgoing Temp(degree),Failure
0,21-09-2021,17:00:00,80,1.8,9.0,8.5,0,16,1,63,...,28,30,0,4.2,4.5,0.33,19,1,15,0
1,21-09-2021,18:00:00,80,1.8,9.0,8.5,0,16,1,63,...,28,30,0,4.2,4.5,0.33,19,1,15,0
2,21-09-2021,19:00:00,80,1.8,9.0,8.5,0,16,1,63,...,28,30,0,4.2,4.5,0.32,19,1,15,0
3,21-09-2021,20:00:00,60,1.8,8.5,8.1,0,15,1,60,...,28,30,0,4.2,4.5,0.30,17,1,15,0
4,21-09-2021,21:00:00,60,1.8,8.5,8.1,0,15,1,60,...,28,30,0,4.2,4.5,0.30,12,1,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,20-08-2022,15:00:00,80,2.0,9.1,8.7,0,15,0,63,...,28,30,75,4.8,4.6,0.32,17,0,11,0
2907,20-08-2022,16:00:00,80,2.0,9.1,8.7,0,15,0,63,...,28,30,75,4.8,4.6,0.32,17,0,11,0
2908,20-08-2022,17:00:00,80,2.0,9.1,8.7,0,15,0,63,...,28,30,75,4.8,4.6,0.32,17,0,11,0
2909,20-08-2022,18:00:00,80,2.0,9.1,8.7,0,15,0,63,...,28,30,75,4.8,4.6,0.32,17,0,11,0


In [45]:
df2 = pd.DataFrame(df)
df2["Date"] = df2['Date'].astype(str) + "  " + df2['Time'].astype(str)
df2['Date']= pd.to_datetime(df2['Date'], format='%d-%m-%Y %H:%M:%S')
df2 = df2.drop(['Time'],axis = 1)
df3=df2
df2.shape

(2911, 27)

In [46]:
sequence_length = 20

# Filter out non-numeric columns
numeric_columns = df2.select_dtypes(include=np.number).columns

# Create empty lists to store the input sequences and corresponding targets
input_sequences = []
targets = []

# Iterate over the dataframe to create input sequences and targets
for i in range(len(df2) - sequence_length):
    sequence = df2.iloc[i:i+sequence_length][numeric_columns].values
    target = df2.iloc[i+sequence_length]['Failure']
    
    input_sequences.append(sequence)
    targets.append(target)

# Convert the input sequences and targets into numpy arrays
input_sequences = np.array(input_sequences)
targets = np.array(targets)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    input_sequences, targets, test_size=0.2, random_state=42)

# Define the input shape
input_shape = (sequence_length, input_sequences.shape[2])

# Define the model
input_layer = keras.Input(shape=input_shape)
query = layers.Dense(64)(input_layer)
value = layers.Dense(64)(input_layer)

attention = layers.Attention()([query, value])
output = layers.Dense(units=1)(attention)

model = keras.Model(inputs=input_layer, outputs=output)

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X_train, y_train, epochs=15, batch_size=256, validation_split=0.2)

predictions = model.predict(X_test)

Epoch 1/15
8/8 [==============================] - 2s 78ms/step - loss: 4373.4272 - val_loss: 718.2453
Epoch 2/15
8/8 [==============================] - 0s 34ms/step - loss: 222.1116 - val_loss: 255.5430
Epoch 3/15
8/8 [==============================] - 0s 33ms/step - loss: 427.1315 - val_loss: 413.4336
Epoch 4/15
8/8 [==============================] - 0s 33ms/step - loss: 235.5431 - val_loss: 37.4453
Epoch 5/15
8/8 [==============================] - 0s 47ms/step - loss: 37.5028 - val_loss: 76.1342
Epoch 6/15
8/8 [==============================] - 0s 32ms/step - loss: 74.5228 - val_loss: 45.9418
Epoch 7/15
8/8 [==============================] - 0s 31ms/step - loss: 29.8067 - val_loss: 25.3359
Epoch 8/15
8/8 [==============================] - 0s 33ms/step - loss: 28.6938 - val_loss: 29.1688
Epoch 9/15
8/8 [==============================] - 0s 31ms/step - loss: 24.7169 - val_loss: 22.1745
Epoch 10/15
8/8 [==============================] - 0s 31ms/step - loss: 21.8583 - val_loss: 23.1326
E

In [47]:
predictions

array([[[ 5.2500796 ],
        [ 5.2500796 ],
        [ 5.2500796 ],
        ...,
        [ 5.2500796 ],
        [ 5.2500796 ],
        [ 5.2500796 ]],

       [[-1.6083457 ],
        [-1.6083457 ],
        [-1.6083457 ],
        ...,
        [-1.6083457 ],
        [-1.6083457 ],
        [-1.6083457 ]],

       [[ 0.14481191],
        [ 0.14481191],
        [ 0.14481191],
        ...,
        [ 0.14481191],
        [ 0.14481191],
        [ 0.14481191]],

       ...,

       [[ 1.2359889 ],
        [ 1.2359889 ],
        [ 1.2359889 ],
        ...,
        [ 1.2359889 ],
        [ 1.2359889 ],
        [ 1.2359889 ]],

       [[ 0.3105891 ],
        [ 0.3105891 ],
        [ 0.3105891 ],
        ...,
        [ 0.3105891 ],
        [ 0.3105891 ],
        [ 0.3105891 ]],

       [[-8.10773   ],
        [-8.10773   ],
        [-8.10773   ],
        ...,
        [-8.10773   ],
        [-8.10773   ],
        [-8.10773   ]]], dtype=float32)

In [62]:
import numpy as np 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

dataset = pd.read_csv('cleaned_data.csv')
df = dataset
df2 = pd.DataFrame(df)
df2["Date"] = df2['Date'].astype(str) + "  " + df2['Time'].astype(str)
df2['Date']= pd.to_datetime(df2['Date'], format='%d-%m-%Y %H:%M:%S')
df2 = df2.drop(['Time'],axis = 1)
df3=df2
df2 = df2.set_index("Date")
data = df2

In [51]:
cols = list(df3)[1:]
cols

['Eqp1 Load(%)',
 'Eqp1 Pressure1(bar)',
 'Eqp1 Pressure2(bar)',
 'Eqp1 Supply Of oil Pressure(bar) ',
 'Eqp1 oil filter variations Presssure(bar)',
 'Eqp1 Temp1(degree)',
 'Eqp1 Temp1 working',
 'Eqp1 Temp2(degree)',
 'Eqp1 oil temp sensor Oil level(mm)',
 'Eqp1 oil temp sensor working',
 'Eqp2 water flow(M/hr)',
 'Eqp2 Incoming Pressure(bar)',
 'Eqp2 OutGoing Pressure(bar)',
 'Eqp2 OutGoing Pressure Working',
 'Eqp2 Pressure(bar) Diff',
 'Eqp2 Pressure Diff Working',
 'Eqp2 Incoming Temp(degree)',
 'Eqp2 Outgoiing Temp(degree)',
 'Eqp3 cold water flow (m/hr)',
 'Eqp3 Incoming Pressure(bar)',
 'Eqp3 Outgoing Pressure(bar)',
 'Eqp3 Variation in Pressure(bar)',
 'Eqp3 Incoming Temp(degree)',
 'Eqp3 Incoming Temp Working',
 'Eqp3 Outgoing Temp(degree)',
 'Failure']

In [57]:
df_for_training = df3[cols].astype(float)
sc = StandardScaler()
scaler = sc.fit(df_for_training)
trained_scaler = scaler.transform(df_for_training)

In [59]:
n_future = 1  # Number of days we want to predict into the future
n_past = 12  # Number of days we want to use to predict the future

trainX = []
trainY = []

for i in range(n_past, len(df_for_training) - n_future + 1):
    trainX.append(df_for_training.iloc[i - n_past:i, :df_for_training.shape[1]])
    trainY.append(df_for_training.iloc[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)
print('trainX shape == {}'.format(trainX.shape))
print('trainY shape == {}'.format(trainY.shape))

trainX shape == (2899, 12, 26)
trainY shape == (2899, 1)


In [63]:
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 12, 64)            23296     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 35,745
Trainable params: 35,745
Non-trainable params: 0
_________________________________________________________________


In [64]:
history = model.fit(trainX,trainY,epochs = 15,batch_size=16,validation_split=0.1,verbose=1)

Epoch 1/15
164/164 [==============================] - 6s 18ms/step - loss: 1211.8533 - val_loss: 56.1666
Epoch 2/15
164/164 [==============================] - 3s 16ms/step - loss: 437.5279 - val_loss: 77.4158
Epoch 3/15
164/164 [==============================] - 3s 15ms/step - loss: 362.5817 - val_loss: 59.3390
Epoch 4/15
164/164 [==============================] - 2s 15ms/step - loss: 238.0461 - val_loss: 103.4718
Epoch 5/15
164/164 [==============================] - 2s 15ms/step - loss: 216.7113 - val_loss: 46.7807
Epoch 6/15
164/164 [==============================] - 3s 17ms/step - loss: 204.9721 - val_loss: 70.7126
Epoch 7/15
164/164 [==============================] - 3s 16ms/step - loss: 211.7492 - val_loss: 43.8875
Epoch 8/15
164/164 [==============================] - 3s 15ms/step - loss: 194.2038 - val_loss: 44.4356
Epoch 9/15
164/164 [==============================] - 3s 17ms/step - loss: 204.7778 - val_loss: 54.2151
Epoch 10/15
164/164 [==============================] - 3s 16ms

In [66]:
# Prepare the input data for prediction
input_sequence = trainX[-1]  # Take the most recent input sequence from the training data

# Forecast 100 days into the future
future_forecast = []
for _ in range(100):
    # Reshape the input sequence to match the model's input shape
    input_sequence = np.reshape(input_sequence, (1, n_past, trainX.shape[2]))

    # Make a prediction using the trained model
    predicted_value = model.predict(input_sequence)

    # Reshape and repeat the predicted value to match the number of features
    predicted_value = np.reshape(predicted_value, (1, 1, 1))
    predicted_value = np.repeat(predicted_value, trainX.shape[2], axis=2)

    # Append the predicted value to the input sequence
    input_sequence = np.concatenate((input_sequence[:, 1:, :], predicted_value), axis=1)

    # Append the predicted value to the future forecast list
    future_forecast.append(predicted_value[0, 0, 0])  # Assuming only one output dimension

# Print the future forecast
print(future_forecast)

1/1 [==============================] - 0s 33ms/step
[76.518394, 50.96991, 56.41936, 72.97264, 76.60534, 84.61263, 88.68535, 100.99024, 134.97424, 193.67307, 275.67264, 355.36258, 460.16006, 542.06793, 701.4106, 873.39667, 1050.3335, 1370.1943, 1778.5431, 2050.908, 2583.6218, 3378.8208, 4029.5076, 5410.525, 6740.307, 8103.0674, 10047.503, 13386.269, 16739.629, 20893.63, 25660.293, 32812.34, 44542.727, 51713.86, 73454.375, 102841.33, 113540.78, 146182.45, 191520.12, 245000.2, 306990.56, 409552.47, 519317.56, 662087.9, 802475.0, 1078223.2, 1300235.0, 1558574.0, 2042315.1, 2734465.5, 3305916.5, 4271290.0, 5157507.0, 6480662.5, 8378041.0, 10441855.0, 14621861.0, 17341616.0, 23115256.0, 27646936.0, 35789816.0, 43184824.0, 59507070.0, 69853240.0, 92490520.0, 117238744.0, 141837630.0, 187718740.0, 250922910.0, 283520860.0, 384147300.0, 493355420.0, 587310900.0, 781215740.0, 927653760.0, 1186143900.0, 1452027100.0, 1945173500.0, 2746357800.0, 3148516900.0, 4063845000.0, 5436837400.0, 6557312000

In [75]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense

# Assuming you have reshaped input data X_train and X_test as 3D arrays
# X_train shape: (samples, time_steps, features)
# X_test shape: (samples, time_steps, features)

# Ensure that the time steps dimension matches (in this case, 12)
trainX = trainX[:, :12, :]
trainX = trainX[:, :12, :]

# Define and compile the CNN model
model1 = Sequential()
model1.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2])))
model1.add(Flatten())
model1.add(Dense(1))

model1.compile(optimizer='adam', loss='mse')

In [78]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
73/73 [==============================] - 0s 3ms/step - loss: 0.2643 - val_loss: 0.2134
Epoch 2/10
73/73 [==============================] - 0s 3ms/step - loss: 0.1523 - val_loss: 0.1445
Epoch 3/10
73/73 [==============================] - 0s 4ms/step - loss: 0.1884 - val_loss: 0.7575
Epoch 4/10
73/73 [==============================] - 0s 4ms/step - loss: 0.1930 - val_loss: 0.2536
Epoch 5/10
73/73 [==============================] - 0s 4ms/step - loss: 0.1432 - val_loss: 0.1374
Epoch 6/10
73/73 [==============================] - 0s 4ms/step - loss: 0.1617 - val_loss: 0.2568
Epoch 7/10
73/73 [==============================] - 0s 4ms/step - loss: 0.2190 - val_loss: 0.3817
Epoch 8/10
73/73 [==============================] - 0s 3ms/step - loss: 0.1967 - val_loss: 0.2722
Epoch 9/10
73/73 [==============================] - 0s 4ms/step - loss: 0.1296 - val_loss: 0.1293
Epoch 10/10
73/73 [==============================] - 0s 4ms/step - loss: 0.1428 - val_loss: 0.1820


In [80]:
predictions = model.predict(X_test)

19/19 [==============================] - 0s 1ms/step


In [92]:
import numpy as np
import xgboost as xgb

# Assuming you have input data X_train and X_test as 3D arrays
# X_train shape: (samples, time_steps, features)
# X_test shape: (samples, time_steps, features)

# Reshape the input data into 2D matrices
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

# Define the XGBoost model
model = xgb.XGBRegressor()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)
predictions

array([-1.53119963e-05, -1.53119963e-05, -1.21400053e-05,  3.74502093e-02,
        1.07384431e-05, -2.14482097e-05,  1.43766438e-05, -1.61800534e-03,
       -2.96713351e-05, -2.14482097e-05,  1.73099082e-08,  1.43766438e-05,
        1.73099082e-08, -9.43640713e-04, -2.14482097e-05, -2.14482097e-05,
       -1.53119963e-05,  1.78361879e-04, -1.15162242e-04,  9.77742002e-06,
        2.05128599e-05, -1.53119963e-05, -8.39676056e-03, -4.58191198e-06,
        1.43766438e-05,  2.05128599e-05,  1.43766438e-05, -2.14482097e-05,
       -2.14482097e-05,  2.05128599e-05, -2.60474299e-05,  4.57388887e-05,
       -1.21400053e-05,  1.73099082e-08, -1.05219797e-05, -4.78656293e-05,
       -4.58191198e-06,  2.05128599e-05, -2.14482097e-05, -3.62089031e-06,
        5.71650053e-05, -3.62089031e-06, -2.14482097e-05, -1.21400053e-05,
       -1.74924171e-05, -1.53119963e-05,  1.43766438e-05,  1.73099082e-08,
        1.73099082e-08, -1.61800534e-03, -2.09464488e-05,  1.43766438e-05,
        2.52023019e-05,  